In [1]:
import cv2
import os
import numpy as np
def load_images_from_directory(directory, target_size=(100, 100)):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)  # Read image using OpenCV
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img_resized = cv2.resize(img_rgb, target_size)  # Resize image
            images.append(img_resized)
    return np.array(images)

In [2]:
blight_path = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Blight'
common_rust_path = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Common_Rust'
gray_leaf_spot_path = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Gray_Leaf_Spot'
healthy_path='/kaggle/input/corn-or-maize-leaf-disease-dataset/data/Healthy'

blight_tensors = load_images_from_directory(blight_path, target_size=(300, 300))
common_rust_tensors=load_images_from_directory(common_rust_path, target_size=(300, 300))
gray_leaf_tensors=load_images_from_directory(gray_leaf_spot_path, target_size=(300, 300))
healthy_tensors= load_images_from_directory(healthy_path, target_size=(300, 300))
print(blight_tensors.shape)
print(common_rust_tensors.shape)

(1146, 300, 300, 3)
(1306, 300, 300, 3)


In [3]:
#labeling healthy as 0, blight as 1, common rust as 2 and gray leaf as 3

In [4]:
x = np.concatenate((healthy_tensors, blight_tensors, common_rust_tensors, gray_leaf_tensors), axis=0)
print(x.shape)

(4188, 300, 300, 3)


In [5]:
num_healthy = healthy_tensors.shape[0]
num_blight = blight_tensors.shape[0]
num_common_rust=common_rust_tensors.shape[0]
num_gray_leaf=gray_leaf_tensors.shape[0]

y_healthy = np.zeros(num_healthy)  # Label 0 for healthy
y_blight = np.ones(num_blight)   # Label 1 for blight
y_common_rust=np.zeros(num_common_rust) #Label 2 for common rust
for i in range(0,len(y_common_rust)):
    y_common_rust[i]=2
y_gray_leaf=np.zeros(num_gray_leaf)#Label 3 for gray leaf
for i in range(0, len(y_gray_leaf)):
    y_gray_leaf[i]=3
y = np.concatenate((y_healthy, y_blight, y_common_rust, y_gray_leaf))  # Concatenate labels

# Shuffle data and labels together
shuffle_indices = np.random.permutation(len(x))
x = x[shuffle_indices]
y = y[shuffle_indices]

In [6]:
import matplotlib.pyplot as plt
#plt.imshow(x[99])
print(y[99])
x = x.astype(np.int8)
y= y.astype(np.int8)

2.0


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2)

In [8]:
import numpy as np
def one_hot_encode(labels, num_classes=4):
    one_hot_labels = np.zeros((labels.size, num_classes))
    one_hot_labels[np.arange(labels.size), labels] = 1
    return one_hot_labels
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot= one_hot_encode(y_test)

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import Xception 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.regularizers import l2

2024-08-08 14:03:33.668897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 14:03:33.669023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 14:03:33.770934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
conv_base=Xception(include_top=False, weights='imagenet', input_shape=(300, 300, 3))

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(restore_best_weights=True, monitor='loss', patience=10)

In [12]:
class Predictor:
    def __init__(self):
        self.model=Sequential()
        self.model.add(conv_base)
        self.model.add(GlobalAveragePooling2D())
        self.model.add(Dropout(0.2))
        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(256, activation='relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dropout(0.2))
        self.model.add(Dense(4, activation='softmax'))
        
    def train(self, x_train, y_train):
        self.model.compile(loss='categorical_crossentropy', optimizer='Adam')
        self.model.fit(x_train, y_train, epochs=500, callbacks=[early_stopping])
        
    def classify(self, x_test):
        return self.model.predict(x_test)
        
    def summarize(self):
        self.model.summary()

In [13]:
predictor=Predictor()
predictor.train(x_train, y_train_one_hot)

Epoch 1/500


I0000 00:00:1723125959.280962     110 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723125959.367856     110 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


104/105 ━━━━━━━━━━━━━━━━━━━━ 0s 764ms/step - loss: 0.7929

W0000 00:00:1723126125.379651     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


105/105 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - loss: 0.7896
Epoch 2/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 765ms/step - loss: 0.4068
Epoch 3/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 760ms/step - loss: 0.2955
Epoch 4/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 761ms/step - loss: 0.2950
Epoch 5/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 757ms/step - loss: 0.2851
Epoch 6/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 756ms/step - loss: 0.2407
Epoch 7/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 759ms/step - loss: 0.2189
Epoch 8/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 759ms/step - loss: 0.2026
Epoch 9/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 757ms/step - loss: 0.2013
Epoch 10/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 756ms/step - loss: 0.2418
Epoch 11/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 760ms/step - loss: 0.1612
Epoch 12/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 760ms/step - loss: 0.1384
Epoch 13/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 760ms/step - loss: 0.1460
Epoch 14/500
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 759ms/step - loss: 0.1440
Epoch 15/500
10

In [14]:
y_pred_one_hot=predictor.classify(x_test)

 1/27 ━━━━━━━━━━━━━━━━━━━━ 1:29 3s/step

W0000 00:00:1723129476.511652     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 525ms/step


W0000 00:00:1723129490.151442     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [15]:
y_pred=np.zeros(y_pred_one_hot.shape[0])
for i in range(0, len(y_pred)):
    y_pred[i]=y_pred_one_hot[i].argmax()

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, y_test)
print(accuracy)

0.9284009546539379
